In [1]:
pip install langchain neo4j openai pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.7/301.7 kB 7.0 MB/s eta 0:00:00


In [ ]:
from neo4j import GraphDatabase

# Neo4j connection details
NEO4J_URI = "neo4j+s://e8bfaf75.databases.neo4j.io"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "your neo4j pass"

# Create a Neo4j graph connection class
class Neo4jGraph:
    def __init__(self, uri, username, password):
        self.driver = GraphDatabase.driver(uri, auth=(username, password))

    def close(self):
        self.driver.close()

    def query(self, query):
        with self.driver.session() as session:
            session.run(query)

# Initialize the connection
graph = Neo4jGraph(NEO4J_URI, NEO4J_USERNAME, NEO4J_PASSWORD)


In [ ]:
import pandas as pd

filename = 'imdb_top_1000.csv'  # Ensure the file is in the same directory
data = pd.read_csv(filename)

# Display the first few rows to verify
print(data.head())


                                         Poster_Link  \
0  https://m.media-amazon.com/images/M/MV5BMDFkYT...   
1  https://m.media-amazon.com/images/M/MV5BM2MyNj...   
2  https://m.media-amazon.com/images/M/MV5BMTMxNT...   
3  https://m.media-amazon.com/images/M/MV5BMWMwMG...   
4  https://m.media-amazon.com/images/M/MV5BMWU4N2...   

               Series_Title Released_Year Certificate  Runtime  \
0  The Shawshank Redemption          1994           A  142 min   
1             The Godfather          1972           A  175 min   
2           The Dark Knight          2008          UA  152 min   
3    The Godfather: Part II          1974           A  202 min   
4              12 Angry Men          1957           U   96 min   

                  Genre  IMDB_Rating  \
0                 Drama          9.3   
1          Crime, Drama          9.2   
2  Action, Crime, Drama          9.0   
3          Crime, Drama          9.0   
4          Crime, Drama          9.0   

                         

In [ ]:
def clean_column(column_data):
    return column_data.apply(lambda x: x.replace("'", "\\'") if pd.notnull(x) else 'NA')

data['Series_Title'] = clean_column(data['Series_Title'])
data['Director'] = clean_column(data['Director'])
data['Certificate'] = clean_column(data['Certificate'])
data['Runtime'] = clean_column(data['Runtime'])
data['Genre'] = clean_column(data['Genre'])
data['IMDB_Rating'] = data['IMDB_Rating'].fillna('NA')
data['Released_Year'] = pd.to_numeric(data['Released_Year'], errors='coerce').fillna(-1)

star_columns = ['Star1', 'Star2', 'Star3', 'Star4']
for column in star_columns:
    data[column] = clean_column(data[column])


In [ ]:
for index, row in data.iterrows():
    movie_title = row['Series_Title']
    director_name = row['Director']
    certificate = row['Certificate']
    runtime = row['Runtime']
    genre = row['Genre']
    imdb_rating = row['IMDB_Rating']
    released_year = int(row['Released_Year']) if row['Released_Year'] != -1 else 'NA'

    # Skip invalid rows
    if released_year == 'NA':
        print(f"Skipping row {index} due to invalid year.")
        continue

    # Cypher queries
    movie_node = (
        f"MERGE (m:Movie {{title: '{movie_title}', year: {released_year}, "
        f"certificate: '{certificate}', runtime: '{runtime}', genre: '{genre}', imdb_rating: {imdb_rating}}})"
    )
    director_node = f"MERGE (d:Director {{name: '{director_name}'}})"
    directed_by_relation = f"MERGE (d)-[:DIRECTED]->(m)"

    # Execute the query
    graph.query(f"{movie_node} {director_node} {directed_by_relation}")


Skipping row 966 due to invalid year.


In [ ]:
for index, row in data.iterrows():
    movie_title = row['Series_Title']
    for s, star in enumerate(star_columns, start=1):
        star_name = row[star]
        if star_name != 'NA':
            star_node = f"MERGE (s{s}:Star {{name: '{star_name}'}})"
            starred_in_relation = f"MERGE (s{s})-[:STARRED_IN]->(m:Movie {{title: '{movie_title}'}})"
            graph.query(f"{star_node} {starred_in_relation}")

In [ ]:
import random

# Define demographic data
birth_years_range = (1950, 2005)
genders = ["Male", "Female", "Other"]
locations = ["USA", "UK", "Canada", "Australia", "France", "Germany", "India", "Japan"]
opinions = ["liked", "loved", "ignored", "disliked", "hated"]

# Create Person nodes
for i in range(1, 101):  # Create 100 persons
    person_name = f"Person {i}"
    birth_year = random.randint(*birth_years_range)
    gender = random.choice(genders)
    location = random.choice(locations)
    person_node = f"CREATE (:Person {{name: '{person_name}', birthYear: {birth_year}, gender: '{gender}', location: '{location}'}})"
    graph.query(person_node)

# Create random relationships with movies
movie_names = data['Series_Title'].tolist()
for _ in range(1000):  # Create 1000 random relationships
    person_index = random.randint(1, 100)
    person_name = f"Person {person_index}"
    movie_name = random.choice(movie_names)
    opinion = random.choice(opinions).upper()
    opinion_query = (
        f"MATCH (m:Movie {{title: '{movie_name}'}}), (p:Person {{name: '{person_name}'}}) "
        f"MERGE (p)-[:{opinion}]->(m)"
    )
    graph.query(opinion_query)

In [ ]:
graph.close()
print("All data has been added to Neo4j.")

All data has been added to Neo4j.


# Implemented with Open AI

In [4]:
pip install -U langchain langchain-neo4j langchain-openai langchain-experimental gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.0/209.0 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.2/320.2 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 85.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 kB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.2/390.2 kB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 107.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 56.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.

In [ ]:
import os
import sys
import logging
os.environ["OPENAI_API_KEY"] = "your api key"
logging.basicConfig(stream=sys.stdout, level=logging.INFO)

In [ ]:
!pip install httpx==0.27.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 2.1 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.0
    Uninstalling httpx-0.28.0:
      Successfully uninstalled httpx-0.28.0


In [ ]:
pip install --force-reinstall -v openai==1.55.3

Using pip 24.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
  Obtaining dependency information for openai==1.55.3 from https://files.pythonhosted.org/packages/10/06/691ef3f0112ecf0d7420d0bf35b5d16cf81554141f4b4913a9831031013d/openai-1.55.3-py3-none-any.whl.metadata
  Obtaining dependency information for anyio<5,>=3.5.0 from https://files.pythonhosted.org/packages/a0/7a/4daaf3b6c08ad7ceffea4634ec206faeff697526421c20f07628c7372156/anyio-4.7.0-py3-none-any.whl.metadata
  Obtaining dependency information for distro<2,>=1.7.0 from https://files.pythonhosted.org/packages/12/b3/231ffd4ab1fc9d679809f356cebee130ac7daa00d6d6f3206dd4fd137e9e/distro-1.9.0-py3-none-any.whl.metadata
  Obtaining dependency information for httpx<1,>=0.23.0 from https://files.pythonhosted.org/packages/2a/39/e50c7c3a983047577ee07d2a9e53faf5a69493943ec3f6a384bdc792deb2/httpx-0.28.1-py3-none-any.whl.metadata
  Obtaining dependency information for jiter<1,>=0.4.0 from https://files.pythonhosted.org/pack

In [ ]:
pip show openai langchain

Name: openai
Version: 1.55.3
Summary: The official Python library for the openai API
Home-page: https://github.com/openai/openai-python
Author: 
Author-email: OpenAI <support@openai.com>
License: Apache-2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: anyio, distro, httpx, jiter, pydantic, sniffio, tqdm, typing-extensions
Required-by: langchain-openai
---
Name: langchain
Version: 0.3.9
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, langchain-core, langchain-text-splitters, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: langchain-community, langchain-neo4j


In [ ]:
from langchain.graphs.neo4j_graph import Neo4jGraph
from langchain.chains import GraphQAChain
from langchain_openai import ChatOpenAI
from langchain.chains import GraphCypherQAChain

# Neo4j connection details
NEO4J_URI = "neo4j+s://e8bfaf75.databases.neo4j.io"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "your neo4j pass"

# Initialize Neo4j connection
neo4j_graph = Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD
)

# Set up the chat model
#llm = ChatOpenAI(temperature=0)

# Create the Graph QA Chain
chain = GraphCypherQAChain.from_llm(ChatOpenAI(temperature=0), graph=neo4j_graph, verbose=True, allow_dangerous_requests=True)


In [ ]:
out1 = chain.run("Which Movies does Person 1 love?")
print(out1)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person {name: 'Person 1'})-[:LOVED]->(m:Movie)
RETURN m.title
Full Context:
[{'m.title': 'Fight Club'}, {'m.title': 'Marriage Story'}, {'m.title': 'Fight Club'}, {'m.title': 'Fight Club'}, {'m.title': 'Fight Club'}, {'m.title': 'Fight Club'}, {'m.title': 'Marriage Story'}, {'m.title': 'Marriage Story'}, {'m.title': 'Marriage Story'}, {'m.title': 'Marriage Story'}]

> Finished chain.
Person 1 loves Fight Club and Marriage Story.


In [ ]:
import gradio as gr

def ask_question(question):
    # Your function to run the question through the chain and return the output
    output = chain.run(question)
    return output

# Corrected Gradio interface setup
iface = gr.Interface(fn=ask_question,
                     inputs=gr.Textbox(lines=2, placeholder="Enter your question here..."),
                     outputs=gr.Textbox(lines=10, label="Output"),  # Increased the lines for the output
                     title="Movie Recomandation System",
                     description="Ask any question related to the movie database.")

# Launch the Gradio app
iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d43cc3550fb1ee840e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# Using Open-Sourced LLM

In [2]:
pip install langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.8/108.8 kB 6.5 MB/s eta 0:00:00


In [5]:
from langchain.graphs.neo4j_graph import Neo4jGraph
from langchain.chains import GraphQAChain
from langchain_groq import ChatGroq
from langchain.chains import GraphCypherQAChain

# Neo4j connection details
NEO4J_URI = "neo4j+s://e8bfaf75.databases.neo4j.io"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "your neo4j pass"

# Initialize Neo4j connection
neo4j_graph = Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD
)

# Set up the chat model using a different model
groq_api_key = "your groq api key"
llm = ChatGroq(groq_api_key=groq_api_key, model_name="llama-3.3-70b-versatile")


<ipython-input-5-09ed25a78219>:12: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  neo4j_graph = Neo4jGraph(


In [6]:
# Create the Graph QA Chain
chain = GraphCypherQAChain.from_llm(llm, graph=neo4j_graph, verbose=True, allow_dangerous_requests=True)
out1 = chain.run("Who are the people that liked Action, Adventure films?")
print(out1)

<ipython-input-6-ec4632e84195>:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  out1 = chain.run("Who are the people that liked Action, Adventure films?")




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person)-[:LIKED]->(m:Movie) 
WHERE m.genre = 'Action, Adventure' 
RETURN p.name
Full Context:
[{'p.name': 'Person 53'}]

> Finished chain.
Person 53 liked Action, Adventure films.


In [7]:
import gradio as gr

def ask_question(question):
    # Your function to run the question through the chain and return the output
    output = chain.run(question)
    return output

# Corrected Gradio interface setup
iface = gr.Interface(fn=ask_question,
                     inputs=gr.Textbox(lines=2, placeholder="Enter your question here..."),
                     outputs=gr.Textbox(lines=10, label="Output"),  # Increased the lines for the output
                     title="Movie Recommendation System",
                     description="Ask any question related to the movie database.")

# Launch the Gradio app
iface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c6fd3ff9ebfdcc3ccf.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
